<a href="https://colab.research.google.com/github/guangyoung/test2/blob/main/test_engine_QG_v10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Javascript
from io import StringIO, BytesIO
import requests
import matplotlib.pyplot as plt
import numpy as np
import json
import math
import yfinance as yf
import datetime
import datetime as dt
import aiohttp
import asyncio
import urllib3
from datetime import timedelta
from scipy import interpolate
import warnings

http = urllib3.PoolManager()

# Global Styling untuk konsistensi
STYLE_W = {'description_width': '117px'}
LAYOUT_W = widgets.Layout(width='335px')
LAYOUT_BUTTONS = widgets.Layout(justify_content='center')

# Data dummy untuk demonstrasi
data_dummy = """
Date,Open,High,Low,Close,Volume
2023-01-01,100,105,99,104,100000
2023-01-02,104,106,103,105,120000
2023-01-03,105,108,104,107,110000
2023-01-04,107,109,106,108,95000
2023-01-05,108,110,107,109,130000
"""
df_data = {}
for i in range(30):
    df_data[f'SAHAM{i+1}'] = pd.read_csv(StringIO(data_dummy))

# 1. Buat widget tampilan
try:
    image_url = 'https://media.istockphoto.com/id/1022809094/vector/concentric-circles-dots-in-circular-form-vector.jpg?s=170667a&w=0&k=20&c=QBPSwryZvuP5-La1oJGzmvEm030m8VIyizAZJ3uO8Hc='
    image_data = requests.get(image_url).content
    ai_logo = widgets.Image(
        value=image_data,
        width=125,
        height=125
    )
except Exception as e:
    print(f"Gagal memuat gambar dari URL. Error: {e}")
    ai_logo = widgets.HTML(
        value="<div style='text-align: center;'>[Gambar AI tidak dapat dimuat]</div>",
        layout=widgets.Layout(width='100px', height='100px', border='1px solid black')
    )

title_text = widgets.HTML(
    value="<p style='text-align: center;line-height: 1.5; font-weight: bold; font-size: 15px; margin-bottom: 10px'>Hi..I'am QuantGenius<br>Test Me With Any Stock Data And See How Accurate My Trade Signals Are</p>"
)

# Konten Accordion "Test Setting"
quantgenius_apikey = widgets.Password(description='QuantGenius API Key:', placeholder='Input QuantGenius API Key', style=STYLE_W, layout=LAYOUT_W)
initial_equity = widgets.FloatText(description='Initial Equity', value=1000000.0, style=STYLE_W, layout=LAYOUT_W)
commission = widgets.FloatText(description='Commission/trade', value=0.001, style=STYLE_W, layout=LAYOUT_W)
spread = widgets.FloatText(description='Spread/trade', value=0.001, style=STYLE_W, layout=LAYOUT_W)
interest_rate = widgets.FloatText(description='Interest Rate/year', value=0.03, style=STYLE_W, layout=LAYOUT_W)
regT_margin_rate = widgets.FloatText(description='Margin Requirement', value=0.5, style=STYLE_W, layout=LAYOUT_W)
maintenance_margin_rate = widgets.FloatText(description='Margin Maintenance', value=0.25, style=STYLE_W, layout=LAYOUT_W)
test_setting_content = widgets.VBox([quantgenius_apikey, initial_equity, commission, spread, interest_rate, regT_margin_rate, maintenance_margin_rate])

# Konten Accordion "Dataset"
#=======================================================================================================================================================================

# --- KONFIGURASI DAN DATA (Bagian 1) ---

try:
    STOCK_DATA = json.loads(requests.get("https://raw.githubusercontent.com/guangyoung/test2/main/yahoo_ticker.json").text)
except:
    STOCK_DATA = {'LSE': ['HSBA.L', 'BP.L'], 'NASDAQ': ['AAPL', 'GOOGL']}

# --- GLOBAL VARIABLES & CONTAINERS ---

CURRENT_UPLOAD_WIDGET = None
OUTPUT_AREA = widgets.Output()
UPLOAD_CONTAINER = widgets.HBox([widgets.Label(value="", layout=widgets.Layout(width='121px'))])

# Data Storage dan Penghitung Baru
UPLOADED_FILES_DATA = {} # Dictionary untuk menyimpan data berkas dari semua sesi upload
TOTAL_FILES_LABEL0 = widgets.Label("0 stock data uploaded")
TOTAL_FILES_LABEL = widgets.HBox([widgets.Label(value="", layout=widgets.Layout(width='122px')), TOTAL_FILES_LABEL0])

# --- WIDGET FACTORIES ---

def create_dropdown(options, value, description, width='180px'):
    return widgets.Dropdown(options=options, value=value, description=description,
                            style=STYLE_W, layout=widgets.Layout(width=width))

def create_slider(value, min_val, max_val, description, is_int=False):
    WidgetType = widgets.IntSlider if is_int else widgets.FloatSlider
    return WidgetType(value=value, min=min_val, max=max_val, description=description,
                      style=STYLE_W, layout=widgets.Layout(width='320px'))

# --- WIDGETS UTAMA DAN BLOK KHUSUS (Bagian 2) ---

# Bagian Kontrol Atas
dataset_dropdown = create_dropdown(list(range(1, 101)), 1, 'Total portfolio to test')
data_need_label = widgets.Label(value="You need 30+ stock data")
dataset_container = widgets.HBox([dataset_dropdown, data_need_label])
data_source_toggle = widgets.Dropdown(
    options=['Yahoo Finance', 'Local Data', 'Monte Carlo Simulation'], value='Yahoo Finance', description='Select Data Source', style=STYLE_W, layout=LAYOUT_W
)
generate_button = widgets.Button(description='Generate Dataset', button_style='primary', layout=widgets.Layout(width='166px'))
reset_button = widgets.Button(description='Reset Dataset', button_style='danger', layout=widgets.Layout(width='166px'))
plot_button = widgets.Button(description='Plot Monte Carlo', button_style='info', icon='bar-chart', layout=widgets.Layout(display='none'))


# Blok 1: Yahoo Finance (Tidak Berubah)
# exchange_dropdown = create_dropdown(list(STOCK_DATA.keys()), 'LSE', 'Select Exchange:')
exchange_dropdown = widgets.Dropdown(options=list(STOCK_DATA.keys()), value='LSE', description='Select Exchange', style=STYLE_W, layout=LAYOUT_W)
select_stock_mode = widgets.Dropdown(options=['Manual', 'Random'], description='Select stock mode', style=STYLE_W, layout=LAYOUT_W)
stock_select_multiple = widgets.SelectMultiple(options=STOCK_DATA['LSE'], value=[], rows=6, description='Select Stocks', style=STYLE_W, layout=LAYOUT_W)
selected_stocks_label = widgets.Label(value="0 stocks selected")
yahoo_price_toggle = widgets.Dropdown(options=['Close', 'Adj Close'], description='Select price type', style=STYLE_W, layout=LAYOUT_W)

stock_selection_container = widgets.VBox([
    stock_select_multiple,
    widgets.HBox([widgets.Label(value="", layout=widgets.Layout(width='122px')), selected_stocks_label])
])

yahoo_widgets = widgets.VBox([exchange_dropdown, select_stock_mode, stock_selection_container, yahoo_price_toggle])


# Blok 2: Local Data (DIPERBAIKI PENEMPATAN WIDGET BARU)
date_col_dropdown = widgets.Dropdown(options=['null'] + list(map(str, range(0, 10))), value='null', description='Date Column Index', style=STYLE_W, layout=widgets.Layout(width='180px'))
price_col_dropdown = widgets.Dropdown(options=['null'] + list(map(str, range(0, 10))), value='null', description='Price Column Index', style=STYLE_W, layout=widgets.Layout(width='180px'))

# Susunan widget lokal: [Upload], [Total Files Label BARU], [Date Column], [Price Column]
local_widgets = widgets.VBox([
    UPLOAD_CONTAINER,
    TOTAL_FILES_LABEL, # WIDGET BARU DITEMPATKAN DI SINI
    date_col_dropdown,
    price_col_dropdown
], layout=widgets.Layout(display='none'))


# Blok 3: Monte Carlo Simulation (Tidak Berubah)
montecarlo_widgets = widgets.VBox([
    create_dropdown(['Geometric Brownian Motion', 'Mean Reversion', 'Jump Diffusion', 'Stochastic Volatility'], 'Geometric Brownian Motion', 'Simulation Method', width='335px'),
    widgets.FloatText(value=100.0, description='Initial Price', style=STYLE_W, layout=widgets.Layout(width='210px')),
    create_slider(0.05, -0.5, 0.5, 'Drift (μ)'),
    create_slider(0.2, 0.01, 1.0, 'Volatility (σ)'),
    create_slider(252, 10, 1000, 'Time Horizon', is_int=True)
], layout=widgets.Layout(display='none'))

main_container = widgets.VBox([yahoo_widgets, local_widgets, montecarlo_widgets])

WIDGET_MAP = {
    'Yahoo Finance': yahoo_widgets,
    'Local Data': local_widgets,
    'Monte Carlo Simulation': montecarlo_widgets
}


# --- HELPER & LOGIC FUNCTIONS (Bagian 3) ---

def update_total_files_label():
    """Memperbarui label total berkas berdasarkan data yang disimpan."""
    TOTAL_FILES_LABEL0.value = f"{len(UPLOADED_FILES_DATA)} stock data uploaded"

def create_upload_widget():
    """Membuat widget FileUpload baru dan menempatkannya di kontainer."""
    global CURRENT_UPLOAD_WIDGET

    new_upload_widget = widgets.FileUpload(
        accept='.csv,.txt', multiple=True, button_style='primary', description='Upload your stock data', layout=widgets.Layout(width='210px')
    )
    new_upload_widget.observe(on_local_data_upload, names='value')

    CURRENT_UPLOAD_WIDGET = new_upload_widget
    UPLOAD_CONTAINER.children = (UPLOAD_CONTAINER.children[0], new_upload_widget)

def on_dataset_update(change):
    min_stock = round(30 * math.sqrt(dataset_dropdown.value))
    data_need_label.set_trait('value', f"You need {min_stock}+ stock data")

def on_local_data_upload(change):
    """Observer untuk FileUpload. Menyimpan data, memproses log, lalu me-reset widget."""
    global UPLOADED_FILES_DATA

    # 1. Simpan data berkas yang baru diunggah ke dictionary global
    # Ini mengakumulasi berkas dari semua sesi upload
    UPLOADED_FILES_DATA.update(change['new'])

    # 2. Perbarui label hitungan
    update_total_files_label()

    # 3. Panggil logika proses/log
    # on_generate_clicked(None, uploaded_files=change['new'])

    # 4. Reset widget FileUpload (untuk membersihkan hitungan di tombol)
    create_upload_widget()

def update_stocks(change):
    """Memperbarui opsi multiple-select saham berdasarkan bursa yang dipilih."""
    stock_select_multiple.options = STOCK_DATA[change['new']]
    stock_select_multiple.value = []

def update_ui(change):
    """Menyembunyikan/menampilkan widget berdasarkan pilihan sumber data."""
    selected_source = change['new']
    for widget in WIDGET_MAP.values():
        widget.layout.display = 'none'

    WIDGET_MAP[selected_source].layout.display = 'block'
    plot_button.layout.display = 'block' if selected_source == 'Monte Carlo Simulation' else 'none'

    if selected_source == 'Yahoo Finance':
        update_stock_dropdown_visibility(select_stock_mode.value)
    elif selected_source == 'Local Data':
        # Pastikan label hitungan diperbarui saat panel diaktifkan
        update_total_files_label()
        if CURRENT_UPLOAD_WIDGET is None or CURRENT_UPLOAD_WIDGET not in UPLOAD_CONTAINER.children:
            create_upload_widget()

def update_stock_dropdown_visibility(value):
    """Menyembunyikan atau menampilkan dropdown Stock berdasarkan mode pemilihan."""
    # stock_selection_container.layout.display = 'block' if value == 'Manual' else 'none'
    if value == 'Random':
      stock_select_multiple.disabled = True
    else:
      stock_select_multiple.disabled = False

# def print_status(actual, minimum, unit):
#     """Fungsi pembantu untuk mencetak status saham/berkas."""
#     if actual < minimum:
#         print(f"⚠️ Silakan sskllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllnpilih setidaknya {minimum} {unit} (Anda baru memilih {actual}).")
#     else:
#         print(f"Jumlah {unit} cukup.")
# Fungsi untuk membersihkan data menggunakan interpolasi linear
def clean_data_simple(df):
    """
    Membersihkan data harga dengan interpolasi linear sederhana.
    Versi yang lebih robust untuk menghindari masalah indexing.
    """
    column="Close"
    threshold_drop=-0.5
    threshold_jump=1.0
    try:
        # Return langsung jika kolom tidak ada atau data pendek
        if column not in df.columns or len(df) < 3:
            return df.copy()

        cleaned = df.copy()
        prices = cleaned[column]

        # Reset index sementara untuk menghindari masalah indexing
        original_index = cleaned.index
        cleaned_reset = cleaned.reset_index(drop=True)
        prices_reset = cleaned_reset[column]

        # Hitung persentase perubahan
        pct_change = prices_reset.pct_change()

        # Cari indices dimana perubahan melebihi threshold
        drop_mask = (pct_change < threshold_drop).to_numpy()
        jump_mask = (pct_change > threshold_jump).to_numpy()
        anomaly_mask = drop_mask | jump_mask

        # Dapatkan indices sebagai array numpy
        anomaly_indices = np.where(anomaly_mask)[0]

        # Iterasi melalui anomaly indices
        for idx in anomaly_indices:
            # Pastikan kita tidak di awal atau akhir data
            if 0 < idx < len(prices_reset) - 1:
                prev_val = prices_reset.iloc[idx-1]
                next_val = prices_reset.iloc[idx+1]

                # Skip jika nilai sebelumnya atau sesudahnya NaN
                if not (np.isnan(prev_val) or np.isnan(next_val)):
                    # Interpolasi linear
                    interpolated_value = (prev_val + next_val) / 2
                    cleaned_reset.at[idx, column] = interpolated_value

        # Kembalikan index asli
        cleaned_reset.index = original_index
        return cleaned_reset

    except Exception as e:
        print(f"Error dalam pembersihan data: {e}")
        import traceback
        traceback.print_exc()
        return df.copy()

def html_head_tail_single_table(df, n=5):
    """
    Menampilkan head dan tail DataFrame dengan lebar kolom otomatis (sesuai teks)
    menggunakan Pandas Styler.
    """

    # 1. Persiapan Data
    df.index = df.index.astype(str)
    head_df = df.head(n)
    tail_df = df.tail(n)
    separator_row = pd.DataFrame([['...'] * len(df.columns)], columns=df.columns, index=['...'])
    combined_df = pd.concat([head_df, separator_row, tail_df])

    # 2. Styling CSS untuk Lebar Otomatis dan Rapi

    # CSS untuk keseluruhan tabel
    # table_css = [
    #     {'selector': 'table',
    #      'props': [
    #         #  ('width', 'auto'), # Mengatur lebar tabel menjadi otomatis
    #          ('table-layout', 'auto') # ⭐️ KUNCI: Biarkan lebar ditentukan oleh konten
    #      ]}
    # ]

    # CSS untuk kolom indeks (tanggal) - Mencegah wrapping teks
    index_css = [
        {'selector': 'th:first-child',
         'props': [
             ('white-space', 'nowrap'), # Mencegah teks melompat ke baris baru
             ('text-align', 'right'),
             ('font-weight', 'bold'),
             ('background-color', '#f7f7f9')
         ]}
    ]

    # CSS untuk semua sel data
    # data_css = [
    #     {'selector': 'td',
    #      'props': [('text-align', 'right'), ('white-space', 'nowrap')]}, # white-space: nowrap juga berguna untuk kolom nilai
    #     {'selector': 'th',
    #      'props': [('text-align', 'center')]}
    # ]

    # 3. Pemformatan Angka (untuk memastikan semua angka memiliki panjang yang seragam)
    # format_mapping = {}
    # for col in combined_df.columns:
    #     if combined_df[col].dtype in ['float64', 'int64']:
    #         format_mapping[col] = "{:,.2f}" # Tetapkan 2 desimal

    # 4. Terapkan Styling dan Formatting
    styler = combined_df.style \
        .set_table_styles(index_css) \
        # .format(format_mapping)

    # 5. Konversi ke HTML dan Tampilkan
    styled_html = styler.to_html()

    html_output = f"""
    <div style="width: 100%; font-family: Arial, sans-serif;">
        <h3 style="color: #2E86AB; text-align: left; margin-bottom: 5px;">
            PORTFOLIO DATASET
        </h3>
        {styled_html}
    </div>
    """

    display(HTML(html_output))

test_data = []
tanggal = []
dataHarga = []

def on_generate_clicked(b, uploaded_files=None):
    """Mencetak ringkasan parameter yang dipilih."""
    with OUTPUT_AREA:
        global dataHarga
        global test_data
        global tanggal

        # print(f"Total Dataset: {dataset_dropdown.value}\nMinimum stocks needed (estimated): {min_stock}\n" + "-" * 40)
        min_stock = round(30 * math.sqrt(dataset_dropdown.value))
        source = data_source_toggle.value

        if source == 'Yahoo Finance':
            num_stocks = len(stock_select_multiple.value)
            print("Sedang proses create dataset......")
            # print_status(num_stocks, min_stock, "saham")

            # # --- Asumsi Variabel Input ---
            # tickers = ("AAME","GABC","ABEO","NEON","ADBE","ADI","ADP","ADSK","HLIT","AGYS","CWCO","AIRT","ALCO","ALOT","GIII","CASH","AMD","AMGN","IIIN","AMWD","MATW","APOG","ARKR","AROW","ARTW","ASRV","ASTE","INCY","AAPL","ATRO")
            tickers = ("BX", "CLPR", "KODK", "VISL", "OIS", "SMBC", "CAC", "VKI", "MAN", "ETJ", "NRIM", "CPS", "BGFV", "BMRA", "MSM", "WTM", "SIG", "INO", "IDXX", "PHD", "RMD", "RFIL", "FBRX", "PLUS", "MLM", "ACHV", "ACR", "NTZ", "XRAY", "GNTY")
            # tickers = ("001065.ks","001067.ks","0011.hk","0012.hk","001230.ks","001440.ks","001465.ks","001520.ks","001525.ks","001550.ks","0016.hk","001620.ks","001680.ks","0017.hk","001740.ks","001790.ks","001800.ks","0019.hk","0021.hk","0023.hk","002350.ks","002460.ks","0027.hk","002790.ks","0028.hk","003075.ks","0033.hk","003470.ks","0035.hk","003545.ks")
            # tickers = stock_select_multiple.value

            # # 1. Tampilkan Indikator Loading untuk Voila
            # status_label = widgets.Label(value="⏳ Sedang mengunduh dan memproses data saham (Mohon tunggu)...")
            # display(status_label)

            # #add data stock
            # portfolioTicker = []
            # portfolioData = []
            # while len(portfolioTicker) < 30:
            #     # print("Portfolio anda masih kurang",30-len(portfolioTicker),"ticker. Mohon isi ticker pilihan anda")
            #     tickerSelect = tickers
            #     for i in range(len(tickerSelect)):
            #         tickerSelect_data = yf.download(tickerSelect[i], period="max", auto_adjust=False)
            #         tickerSelect_data.index = pd.to_datetime(tickerSelect_data.index).date
            #         if len(tickerSelect_data) > 1000:
            #             portfolioTicker.append(tickerSelect[i])
            #             portfolioData.append(tickerSelect_data)

            # Daftar ticker
            # tickers = (
            #     "BX", "CLPR", "KODK", "VISL", "OIS", "SMBC", "CAC", "VKI", "MAN", "ETJ",
            #     "NRIM", "CPS", "BGFV", "BMRA", "MSM", "WTM", "SIG", "INO", "IDXX", "PHD",
            #     "RMD", "RFIL", "FBRX", "PLUS", "MLM", "ACHV", "ACR", "NTZ", "XRAY", "GNTY"
            # )

            # ----------------------------
            # Bagian pengunduhan & pembersihan data
            # ----------------------------

            portfolioTicker = []
            portfolioData = []

            for ticker in tickers:
                # try:
                    # Unduh data historis
                tickerSelect_data = yf.download(ticker, period="max", auto_adjust=False, progress = False)

                # Konversi index ke format tanggal
                tickerSelect_data.index = pd.to_datetime(tickerSelect_data.index).date

                # Cek jumlah minimal data
                if len(tickerSelect_data) > 1000:

                    # Bersihkan kolom Close
                    # tickerSelect_data = clean_data_simple(
                    #     tickerSelect_data
                    # )

                    portfolioTicker.append(ticker)
                    portfolioData.append(tickerSelect_data)
                    # print(f"Data {ticker} (jumlah: {len(tickerSelect_data)}) berhasil diunduh dan dibersihkan.")
                # else:
                #     print(f"Data {ticker} dilewati: Jumlah baris ({len(tickerSelect_data)}) kurang dari 1000.")

                # except Exception as e:
                #     print(f"Gagal mengunduh data {ticker}: {e}")

            # Find common date range
            testStartdate = max([data.index.min() for data in portfolioData])
            testEnddate = min([data.index.max() for data in portfolioData])

            # Create common date range (hanya hari kerja)
            common_dates = pd.date_range(start=testStartdate, end=testEnddate, freq='B').date  # 'B' untuk Business days

            # Filter out weekends from each portfolio data
            portfolioData_weekdays = []
            for data in portfolioData:
                # Konversi index ke datetime untuk filtering
                data_index_dt = pd.to_datetime(data.index)
                # Filter hanya hari kerja (Senin-Jumat)
                weekday_mask = data_index_dt.dayofweek < 5  # 0=Senin, 4=Jumat, 5=Sabtu, 6=Minggu
                data_weekdays = data[weekday_mask]
                portfolioData_weekdays.append(data_weekdays)

            # Combine all data
            combined_data = pd.DataFrame(index=common_dates)

            for i, (ticker, data) in enumerate(zip(portfolioTicker, portfolioData_weekdays)):
                combined_data[ticker] = data['Close'].reindex(common_dates)

            # Handle missing values
            combined_data.ffill(inplace=True)
            combined_data.bfill(inplace=True)

            # Final data - pastikan hanya hari kerja yang tersisa
            test_data = combined_data.dropna()  # Remove any remaining NaN

            # Verifikasi bahwa tidak ada data weekend
            final_dates = pd.to_datetime(test_data.index)
            weekend_mask = final_dates.dayofweek >= 5
            if weekend_mask.any():
                print("Peringatan: Masih terdapat data weekend, melakukan pembersihan tambahan...")
                test_data = test_data[~weekend_mask]

            dataHarga = test_data.to_numpy()
            tanggal = test_data.index.to_numpy()

            clear_output(wait=True)

            html_head_tail_single_table(test_data, 5)
            # print("Your Portfolio Data:\n",test_data)

        elif source == 'Local Data':
            # Ambil total berkas yang sudah diakumulasi
            num_files = len(UPLOADED_FILES_DATA)
            # print(f"Sumber Data: Local Data (Total stock data uploaded: {num_files})")
            # print_status(num_files, min_stock, "berkas")

            print("\nDaftar Berkas yang Diunggah (Total Akumulasi):")
            # Iterasi menggunakan UPLOADED_FILES_DATA global
            print("\n".join(f"- {name}" for name in UPLOADED_FILES_DATA.keys()) if UPLOADED_FILES_DATA else "- Belum ada berkas yang diunggah.")

        elif source == 'Monte Carlo Simulation':
            method = montecarlo_widgets.children[0].value
            print(f"Melakukan simulasi Monte Carlo dengan metode {method}...")

def plot_montecarlo(b):
    """Membuat plot sederhana dari simulasi Monte Carlo."""
    if data_source_toggle.value != 'Monte Carlo Simulation':
        print("Pilih 'Monte Carlo Simulation' untuk melihat plot.")
        return

    initial_price = montecarlo_widgets.children[1].value
    time_horizon = montecarlo_widgets.children[4].value

    plt.figure(figsize=(10, 5))
    for _ in range(50):
        prices = initial_price * np.exp(np.cumsum(np.random.normal(0, 0.01, time_horizon)))
        plt.plot(prices, alpha=0.5)
    plt.title('Monte Carlo Simulation Sample Paths')
    plt.xlabel('Time (days)')
    plt.ylabel('Price')
    plt.grid(True)
    plt.show()

# --- INITIALIZATION & DISPLAY ---

# Link observers
data_source_toggle.observe(update_ui, names='value')
dataset_dropdown.observe(on_dataset_update, names='value')
select_stock_mode.observe(lambda change: update_stock_dropdown_visibility(change['new']), names='value')
exchange_dropdown.observe(update_stocks, names='value')
stock_select_multiple.observe(lambda change: selected_stocks_label.set_trait('value', f"{len(change['new'])} stocks selected"), names='value')
generate_button.on_click(on_generate_clicked)
plot_button.on_click(plot_montecarlo)

# Initial setup
create_upload_widget()

dataset_content = widgets.VBox([
        dataset_container,
        data_source_toggle,
        main_container,
        widgets.HBox([generate_button,reset_button])
    ], layout=widgets.Layout(width='350px'))

#=======================================================================================================================================================================

# file = open("https://images.ctfassets.net/hrltx12pl8hq/28ECAQiPJZ78hxatLTa7Ts/2f695d869736ae3b0de3e56ceaca3958/free-nature-images.jpg?fit=fill&w=1200&h=630", "rb")
image = requests.get("https://raw.githubusercontent.com/guangyoung/test2/refs/heads/main/360_F_1394167414_FkutDkFwLRQETxtp7PdrmBWBYMGoezIY.jpg").content
imge = widgets.Image(
    value=image,
    format='jpg',
    width=745
)

image2 = requests.get("https://raw.githubusercontent.com/guangyoung/test2/refs/heads/main/360_F_1394167414_FkutDkFwLRQETxtp7PdrmBWBYMGoezIY_copy.jpg").content
imge2 = widgets.Image(
    value=image2,
    format='jpg',
    width=745
)

# dataset = widgets.HBox([imge, dataset_content])
dataset = widgets.VBox([widgets.HBox([dataset_content, widgets.VBox([imge])], layout=widgets.Layout(width='1100px')),widgets.HBox([OUTPUT_AREA])])
test_setting = widgets.HBox([test_setting_content, widgets.VBox([imge2])], layout=widgets.Layout(width='1100px'))

# Gabungkan konten ke dalam satu widget Accordion tunggal
main_accordion = widgets.Accordion(children=[test_setting, dataset],layout=widgets.Layout(width='95%'))
# main_accordion.set_title(0, 'QuantGenius API Key')
main_accordion.set_title(0, 'Test Setting')
main_accordion.set_title(1, 'Dataset')

# Set accordion agar tertutup saat pertama kali dimuat
main_accordion.selected_index = None

# Tombol "Start Simulation Trading"
start_button = widgets.Button(
    description='▶️ Start Trade Simulation With Real-Time QuantGenius Trade Signals',
    button_style='success',
    tooltip='Click to run the simulation',
    layout=widgets.Layout(width='95%'),
    style={'font_weight': 'bold', 'font_size': '15px'}
)

# Gunakan HBox untuk menempatkan tombol di tengah
# centered_button = widgets.VBox(
#     [main_accordion,start_button],
#     layout=widgets.Layout(align_items='center')
# )
# Tambahkan tombol reset di luar fungsi simulasi
reset2_button = widgets.Button(
    description="🔄 Reset Trade Simulation",
    button_style='warning',
    tooltip='Hapus hasil simulasi',
    layout=widgets.Layout(width='100%'),
    style={'font_weight': 'bold', 'font_size': '15px'}
)
centered_reset_button = widgets.VBox(
    [reset2_button],layout=widgets.Layout(align_items='center', overflow='hidden')
)

# Wadah untuk menampilkan hasil simulasi (grafik, dll.)
simulation_output = widgets.Output(layout=widgets.Layout(width='95%'))

# def open_url(url):
#     """
#     Fungsi ini akan mengembalikan handler (fungsi yang menerima objek tombol)
#     yang menjalankan kode JavaScript untuk membuka URL tertentu.
#     """
#     def handler(b):
#         # Gunakan URL yang sudah "tertanam" di fungsi ini
#         js_code = f"window.open('{url}', '_blank');"
#         display(Javascript(js_code))
#     return handler

# Tambahkan tombol dan badge baru
github_badge = widgets.HTML(
    f"""
    <a href="{'https://www.github.com'}" target="_blank" style="text-decoration:none;">
        <button class="widget-button" style="width:170px;margin:5px;background-color:lightblue;border:none;padding:3px 12px;border-radius:6px;cursor:pointer;font-size:12px;"><i class="fa fa-github"></i> Github Profile</button>
    </a>
    """
)
# about_badge = widgets.Button(
#     description='About QuantGenius',
#     icon='book',
#     layout=widgets.Layout(width='175px', margin='5px'),
#     style={'button_color': 'lightblue'}
# )
about_badge = widgets.HTML(
    f"""
    <a href="{'https://www.quantgenius.ai'}" target="_blank" style="text-decoration:none;">
        <button class="widget-button" style="width:170px;margin:5px;background-color:lightblue;border:none;padding:3px 12px;border-radius:6px;cursor:pointer;font-size:12px;"><i class="fa fa-book"></i> About QuantGenius</button>
    </a>
    """
)
open_in_googlecolab = widgets.HTML(
    f"""
    <a href="{'https://colab.research.google.com'}" target="_blank" style="text-decoration:none;">
        <button class="widget-button" style="width:170px;margin:5px;background-color:lightblue;border:none;padding:3px 12px;border-radius:6px;cursor:pointer;font-size:12px;"><i class="fa fa-book"></i> Open in Google Colab</button>
    </a>
    """
)
api_doc = widgets.HTML(
    f"""
    <a href="{'https://editor.swagger.io/?_gl=1*1ua8uzl*_gcl_au*NDAxMzAyMjg5LjE3NTc1OTU0NTk.'}" target="_blank" style="text-decoration:none;">
        <button class="widget-button" style="width:170px;margin:5px;background-color:lightblue;border:none;padding:3px 12px;border-radius:6px;cursor:pointer;font-size:12px;"><i class="fa fa-book"></i> QuantGenius API Doc</button>
    </a>
    """
)

# github_badge.on_click(open_url('https://www.github.com/'))
# about_badge.on_click(open_url('https://www.quantgenius.ai/'))
# open_in_googlecolab.on_click(open_url('https://colab.research.google.com/'))
# api_doc.on_click(open_url('https://editor.swagger.io/?_gl=1*1ua8uzl*_gcl_au*NDAxMzAyMjg5LjE3NTc1OTU0NTk.'))

# button = widgets.HTML(f"""
#     <a href="{'https://colab.research.google.com/'}" target="_blank">
#         <button style="padding:8px 16px; font-size:14px;">Open Google</button>
#     </a>
# """)

# Gunakan HBox untuk menempatkan badge di tengah
centered_badges = widgets.HBox(
    [github_badge, open_in_googlecolab, api_doc]
)

# async def resetData(url,api):
#     async with aiohttp.ClientSession() as session:
#         async with session.delete(url, headers = {'X-QUANTGENIUS-APIKEY':api, 'Content-Type': 'application/json'}) as resp:
#              return await resp.text()

# 2. Logika simulasi
# CATATAN: Kode ini mengasumsikan variabel global seperti 'simulation_output',
# 'main_accordion', 'initial_equity', 'interest_rate', 'maintenance_margin_rate',
# 'regT_margin_rate', 'commission', dan 'spread' telah didefinisikan di lingkungan Anda.

# =================================================================================
## FUNGSI PERHITUNGAN METRIK
# =================================================================================

def calculate_metrics(equity_array, initial_equity_value, risk_free_rate=0.02):
    """Menghitung Total Return, Max Drawdown, Sharpe Ratio, dan Sortino Ratio."""

    if len(equity_array) == 0:
        return {"Total Return": np.nan, "Max Drawdown": np.nan, "Sharpe Ratio": np.nan, "Sortino Ratio": np.nan}

    total_return = (equity_array[-1] - initial_equity_value) / initial_equity_value
    daily_returns = pd.Series(equity_array).pct_change().dropna().values

    # Max Drawdown
    cumulative_return = np.insert(equity_array / initial_equity_value, 0, 1.0)
    peak = np.maximum.accumulate(cumulative_return)
    drawdown = (peak - cumulative_return) / peak
    max_drawdown = np.max(drawdown)

    if len(daily_returns) < 1:
        return {"Total Return": total_return, "Max Drawdown": max_drawdown, "Sharpe Ratio": np.nan, "Sortino Ratio": np.nan}

    annual_risk_free_rate = risk_free_rate / 252
    avg_daily_return = np.mean(daily_returns)
    std_daily_return = np.std(daily_returns)

    # Sharpe Ratio
    if std_daily_return == 0:
        sharpe_ratio = np.inf if avg_daily_return > annual_risk_free_rate else np.nan
    else:
        sharpe_ratio = (avg_daily_return - annual_risk_free_rate) / std_daily_return * np.sqrt(252)

    # Sortino Ratio
    downside_returns = daily_returns[daily_returns < 0]
    if len(downside_returns) == 0:
        sortino_ratio = np.inf if avg_daily_return > annual_risk_free_rate else np.nan
    else:
        downside_deviation = np.std(downside_returns)
        if downside_deviation == 0:
            sortino_ratio = np.inf if avg_daily_return > annual_risk_free_rate else np.nan
        else:
            sortino_ratio = (avg_daily_return - annual_risk_free_rate) / downside_deviation * np.sqrt(252)

    return {"Total Return": total_return, "Max Drawdown": max_drawdown, "Sharpe Ratio": sharpe_ratio, "Sortino Ratio": sortino_ratio}

# =================================================================================
## FUNGSI SIMULASI YANG DIPERBAIKI (LEBIH CEPAT)
# =================================================================================

def start_simulation(test_data, dataHarga, tanggal):

    # Clear previous output
    clear_output(wait=True)

    if len(test_data) < 1000:
        #Menampilkan peringatan (Warning) dan melanjutkan eksekusi (jika memungkinkan)
        # warnings.warn("Perhatian: Jumlah data kurang dari 1000. Akurasi dan validitas simulasi mungkin terganggu.", UserWarning)
        raise ValueError("Data input tidak memadai. Diperlukan minimal 1000 titik data untuk menjalankan simulasi.")

    reset = http.request("DELETE", "https://www.quantgenius.ai/api/resetdata", headers = {"X-QUANTGENIUS-APIKEY":"3fJ50FlM4Qsb53p548FmSEoFlv7FbeNwSY3hWIVs"})
    print(reset.json())

    # 1. Inisialisasi Progress Bar
    total_iterations = len(test_data)
    progress_bar = widgets.IntProgress(
        value=0,
        min=0,
        max=total_iterations,
        description='Running trade simulation',
        bar_style='info',  # Gaya 'info' (biru)
        style={'bar_color': 'green', 'description_width': '145px'},
        layout=widgets.Layout(width='94%')
    )
    progress_label = widgets.Label(value=f"0.00%")
    progress_container = widgets.HBox([progress_bar, progress_label])

    with simulation_output:
        # main_accordion.selected_index = None
        start_button.disabled = True
        # Clear previous output
        clear_output(wait=True)
        # Tampilkan progress bar sebelum loop dimulai
        display(progress_container)

        # 2. Inisialisasi Variabel
        stock_price = np.zeros(30, dtype='d')
        stock_price_sebelum = np.zeros(30, dtype='d')
        stock_position_size = np.zeros(30, dtype='d')

        initial_stockPrice = np.array(dataHarga[0], dtype='d')
        initial_eq = initial_equity.value
        cash_balance = initial_eq

        # GANTI np.array APPEND DENGAN LIST APPEND (LEBIH CEPAT)
        quantxi_equity_list = []
        buyhold_equity_list = []
        market_value_list = []

        previous_signal_hashcode = "0000000000000000000000000000000000000000000000000000000000000000"
        dataIdx = 0
        accrued_interest = 0
        min_excessLiquidity = initial_eq

        stock_price_sebelum = dataHarga[dataIdx]

        # 3. Loop Simulasi
        while dataIdx < total_iterations:

            # --- Perbarui Progress Bar ---
            progress_bar.value = dataIdx + 1
            progress_label.value = f"{((dataIdx + 1)/total_iterations)*100:.2f}%"
            # -----------------------------

            stock_price = dataHarga[dataIdx]

            # if dataIdx < total_iterations-1:
            #   stock_price_sesudah = dataHarga[dataIdx+1]
            # else:
            #   stock_price_sesudah = dataHarga[dataIdx]

            # for i in range(30):
            #    if dataIdx == (total_iterations-1):
            #       selisihawal = dataHarga[dataIdx][i] / dataHarga[0][i]
            #       print(f"selisihawal: {selisihawal}")
            #    selisihharga = stock_price[i] / stock_price_sebelum[i]
            #    if selisihharga > 2:
            #       # stock_price[i] = stock_price_sebelum[i] * 2
            #       print(f"selisihharga: {selisihharga}, idx : {dataIdx}")
            #    elif selisihharga < 0.5:
            #       # stock_price[i] = stock_price_sebelum[i] *0.5
            #       print(f"selisihharga: {selisihharga}, idx : {dataIdx}")


            # Logika Bunga
            if cash_balance < 0:
                # print((tanggal[dataIdx] - tanggal[dataIdx-1]).days)
                daily_Interest = cash_balance * (interest_rate.value / 360) * (tanggal[dataIdx] - tanggal[dataIdx-1]).days
                accrued_interest += daily_Interest

            if tanggal[dataIdx].day == 1:
                cash_balance += accrued_interest
                accrued_interest = 0

            market_value = np.sum(stock_price * stock_position_size)
            equity_with_loanValue = cash_balance + market_value + accrued_interest

            # Persiapan Data Input API (Menggunakan list comprehension yang ringkas)
            portfolio_data = [{"price": stock_price[i], "position_size": stock_position_size[i]} for i in range(30)]
            data_input = {
                "prev_signal_hashcode": previous_signal_hashcode,
                "equity_balance": equity_with_loanValue,
                "portfolio_data": portfolio_data
            }
            # print(data_input)

            # PANGGILAN API (Menggunakan requests yang sudah diimpor)
            # try:
            response = http.request("POST", "https://www.quantgenius.ai/api/adddata", headers = {"X-QUANTGENIUS-APIKEY":"3fJ50FlM4Qsb53p548FmSEoFlv7FbeNwSY3hWIVs", "Content-Type": 'application/json'}, json = data_input)
            # response = await addData(json.dumps(data_input))
            # res = json.loads(response)
            # print(json.loads(response.data))
            # response = requests.post("https://api.quantgenius.ai/add", headers = {"X-API-KEY":api_key.value, "Content-Type": 'application/json'}, data = json.dumps(data_input))

            signal_output = response.json()["data"]#json.loads(response.data)["data"]#INI DIPIKIRIN JIKA STATUS SIGNAL TIDAK SUCCESS, KADANG MUNCUL KEY ERROR DATA, yg artinya key nya bisa jd message, ada error saat rest api

            previous_signal_hashcode = signal_output["signal_hashcode"]

            # ----------------------------------------------------------------------------------
            # TRADE TRANSACTION (Dioptimasi dengan Vektorisasi NumPy)
            # ----------------------------------------------------------------------------------
            maintenance_margin_req = market_value * maintenance_margin_rate.value
            excess_liquidity = equity_with_loanValue - maintenance_margin_req
            regT_margin_req = market_value * regT_margin_rate.value
            excess_equity = equity_with_loanValue - regT_margin_req

            if excess_liquidity > 0:
                # Inisialisasi variabel
                estimate_imr = 0
                estimate_comm = 0
                filled_percentage = 0

                # Loop pertama untuk menghitung estimate_imr dan estimate_comm
                for i in range(30):
                    action = signal_output["trade_signal"][i]["action"]
                    quantity = float(signal_output["trade_signal"][i]["quantity"])
                    price = stock_price[i]

                    if action == "BUY":
                        estimate_imr += (quantity * price) * regT_margin_rate.value
                        estimate_comm += abs(quantity) * price * (commission.value + spread.value)
                    elif action == "SELL":
                        estimate_imr -= (quantity * price) * regT_margin_rate.value
                        estimate_comm += abs(quantity) * price * (commission.value + spread.value)
                    else:
                        # Tidak melakukan apa-apa untuk action selain BUY/SELL
                        pass

                # Menghitung filled_percentage
                total_estimate = estimate_imr + estimate_comm

                if total_estimate <= 0:
                    filled_percentage = 1
                elif excess_equity <= 0:
                    filled_percentage = 0
                elif excess_equity > total_estimate:
                    filled_percentage = 1
                else:
                    filled_percentage = excess_equity / total_estimate

                # Inisialisasi array
                filledOrder = []
                filledPrice = []
                tradeValue = []
                commission_arr = []

                # Loop kedua untuk menghitung order yang terisi
                for i in range(30):
                    action = signal_output["trade_signal"][i]["action"]
                    quantity = float(signal_output["trade_signal"][i]["quantity"])
                    price = stock_price[i]

                    if action == "BUY":
                        filledOrder.append(quantity * filled_percentage)
                        filledPrice.append(price)
                    elif action == "SELL":
                        filledOrder.append(-(quantity * filled_percentage))
                        filledPrice.append(price)
                    else:
                        filledOrder.append(0)
                        filledPrice.append(0)

                    # Hitung trade value dan commission
                    trade_value_item = filledOrder[i] * filledPrice[i]
                    commission_item = abs(filledOrder[i]) * filledPrice[i] * (commission.value + spread.value)

                    tradeValue.append(trade_value_item)
                    commission_arr.append(commission_item)

                # Hitung total
                total_trade_value = sum(tradeValue)
                total_commission = sum(commission_arr)

                # Update Cash dan Posisi
                cash_balance -= total_trade_value + total_commission
                stock_position_size += filledOrder

            # ----------------------------------------------------------------------------------
            # PENGUMPULAN DATA
            # ----------------------------------------------------------------------------------
            # market_value = np.sum(stock_price * stock_position_size)
            # equity_with_loanValue = cash_balance + accrued_interest + market_value

            stock_price_sebelum = dataHarga[dataIdx]

            quantxi_equity_list.append(equity_with_loanValue)

            # Buy & Hold Equity (Vektorisasi Cepat)
            bh_position = initial_eq / 30 / initial_stockPrice
            buyhold_equity = np.sum(bh_position * stock_price)
            buyhold_equity_list.append(buyhold_equity)

            market_value_list.append(market_value)

            # if (market_value/equity_with_loanValue)>2:
            #     print(market_value/equity_with_loanValue)

            if excess_liquidity < min_excessLiquidity:
                min_excessLiquidity = excess_liquidity

            dataIdx += 1

        # 1. Membuat data yang merepresentasikan tabel
        data = {
            'SYSTEM': ['QUANTGENIUS', 'BUY AND HOLD'],
            'TOTAL RETURN': ['10,000,000.00%', '10,000.00%'],
            'CAGR': ['100.00%', '5.00%'],
            'MAXDD ABSOLUT': ['65.00%', '55.00%'],
            'MAXDD RELATIF': ['25.00%', '55.00%'],
            'CALMAR ABSOLUT': ['1.54', '0.09'],
            'CALMAR RELATIF': ['4.00', '0.09'],
            'SHARPE RATIO': ['0.80', '0.60'],
            'SORTINO RATIO': ['1.10', '0.70']
        }

        df = pd.DataFrame(data)

        # 2. Mengubah nama kolom dengan HTML line breaks untuk header 2 baris
        df.columns = [
            'SYSTEM',
            'TOTAL<br>RETURN',
            'CAGR',
            'MAX DD<br>ABSOLUT',
            'MAX DD<br>RELATIF',
            'CALMAR<br>ABSOLUT',
            'CALMAR<br>RELATIF',
            'SHARPE<br>RATIO',
            'SORTINO<br>RATIO'
        ]

        # 3. Menampilkan judul
        # print("## PERFORMANCE COMPARISON: QUANTGENIUS vs BUY AND HOLD")

        # 4. Styling tabel dengan HTML formatting
        styled_df = df.style \
            .set_properties(**{
                # 'text-align': 'center',
                'border': '1px solid black',
                'padding': '8px',
                'font-size': '12px',
                'white-space': 'pre-wrap'  # Penting untuk line breaks
            }) \
            .set_table_styles([
                # Styling untuk header
                {'selector': 'th',
                'props': [
                    ('background-color', '#2E8B57'),
                    ('color', 'white'),
                    ('font-weight', 'bold'),
                    ('border', '1px solid black'),
                    ('padding', '12px 8px'),
                    ('text-align', 'center'),
                    ('vertical-align', 'middle'),
                    ('font-size', '12px')
                    #  ('white-space', 'normal')  # Memungkinkan line breaks
                ]},
                # Styling untuk sel data
                {'selector': 'td',
                'props': [
                    ('border', '1px solid black'),
                    ('padding', '8px'),
                    ('text-align', 'right'),
                    ('vertical-align', 'middle')
                ]},
                # Styling untuk baris ganjil
                {'selector': 'tr:nth-child(odd)',
                'props': [('background-color', '#F0FFF0')]},
                # Styling untuk baris genap
                {'selector': 'tr:nth-child(even)',
                'props': [('background-color', '#FFFFFF')]},
                # Styling untuk tabel
                {'selector': 'table',
                'props': [
                    ('border-collapse', 'collapse'),
                    ('width', '100%'),
                    ('margin', '20px 0')
                ]}
            ]) \
            .hide(axis="index") \
            .map(lambda x: 'font-weight: bold; color: #006400;',
                subset=pd.IndexSlice[:, ['SYSTEM']]) \

        html_code = """
          <div style="
              display: flex;
              justify-content: center;
              margin: 20px 0;
          ">
              <div style="
                  text-align: center;
                  padding: 20px 40px;
                  background: linear-gradient(135deg, #E8F5E9, #A5D6A7);
                  border: 2px solid #2E8B57;
                  border-radius: 12px;
                  box-shadow: 0 4px 12px rgba(46, 139, 87, 0.3);
              ">
                  <div style="
                      font-size: 18px;
                      font-weight: bold;
                      color: #1B5E20;
                      margin-bottom: 5px;
                  ">
                      Outperform Total Return
                  </div>
                  <div style="
                      font-size: 42px;
                      font-weight: bold;
                      color: #006400;
                      margin-top: 5px;
                  ">
                      125%
                  </div>
              </div>
          </div>
          """

        # 4. Finalisasi, Metrik, dan Grafik
        # Konversi List ke Array NumPy
        quantxi_equity_array = np.array(quantxi_equity_list, dtype='d')
        buyhold_equity_array = np.array(buyhold_equity_list, dtype='d')
        market_value_array = np.array(market_value_list, dtype='d')

        # Hitung Metrik
        metrics_qg = calculate_metrics(quantxi_equity_array, initial_eq)
        metrics_bh = calculate_metrics(buyhold_equity_array, initial_eq)

        # clear_output(wait=True) # Hapus progress bar final

        # --- KONSTANTA ---
        WINDOW = 20
        ANNUAL_TRADING_DAYS = 252
        RISK_FREE_RATE = 0.0 # Asumsi Rf harian = 0

        # --- VISUALISASI GRAFIK ---
        dates = pd.to_datetime(tanggal[:total_iterations])
        qg_returns = (quantxi_equity_array - initial_eq) / initial_eq
        bh_returns = (buyhold_equity_array - initial_eq) / initial_eq

        # Ubah subplot menjadi 8 baris
        fig, (ax1, ax3, ax8) = plt.subplots(3, 1, figsize=(12, 9), sharex=True)

        # Tambahkan jarak vertikal antar plot untuk memberi ruang pada Catatan
        plt.subplots_adjust(hspace=8.5)

        # ----------------------------------------------------------------------------------
        # Plot 1: Return Kumulatif
        # ----------------------------------------------------------------------------------
        ax1.plot(dates, qg_returns * 100, label='QuantGenius', color='blue')
        ax1.plot(dates, bh_returns * 100, label='Buy & Hold', color='red')
        ax1.set_title('Perbandingan Total Return Kumulatif', fontsize=14)
        ax1.set_ylabel('Return (%)', fontsize=10)
        ax1.tick_params(axis='x', rotation=45)
        ax1.legend(loc='upper left')
        ax1.grid(True, linestyle='--', alpha=0.6)
        ax1.text(0.0, -0.2,
                'Catatan: Menunjukkan pertumbuhan modal dari waktu ke waktu. QG (biru) dibandingkan B&H (merah).\nMenunjukkan pertumbuhan modal dari waktu ke waktu. QG (biru) dibandingkan B&H (merah).\nMenunjukkan pertumbuhan modal dari waktu ke waktu. QG (biru) dibandingkan B&H (merah).',
                transform=ax1.transAxes, fontsize=9, color='black', ha='left')

        # ----------------------------------------------------------------------------------
        # --- PERHITUNGAN VOLATILITAS, RETURNS & RASIO ---
        # ----------------------------------------------------------------------------------

        # Log Returns Harian
        qg_log_returns = pd.Series(np.diff(np.log(quantxi_equity_array)))
        bh_log_returns = pd.Series(np.diff(np.log(buyhold_equity_array)))

        # Volatilitas Bergulir Harian (StDev)
        qg_rolling_stdev = qg_log_returns.rolling(window=WINDOW).std()
        bh_rolling_stdev = bh_log_returns.rolling(window=WINDOW).std()

        # 1. Rolling Annualized Volatility (untuk Plot 7)
        qg_rolling_volatility = qg_rolling_stdev * np.sqrt(ANNUAL_TRADING_DAYS)
        bh_rolling_volatility = bh_rolling_stdev * np.sqrt(ANNUAL_TRADING_DAYS)

        # 2. Rolling Annualized Mean Return
        qg_rolling_mean_ret = qg_log_returns.rolling(window=WINDOW).mean() * ANNUAL_TRADING_DAYS
        bh_rolling_mean_ret = bh_log_returns.rolling(window=WINDOW).mean() * ANNUAL_TRADING_DAYS

        # 3. Rolling Sharpe Ratio (untuk Plot 5)
        qg_rolling_sharpe = (qg_rolling_mean_ret - RISK_FREE_RATE) / qg_rolling_volatility
        bh_rolling_sharpe = (bh_rolling_mean_ret - RISK_FREE_RATE) / bh_rolling_volatility

        # 4. Rolling Sortino Ratio (untuk Plot 6)
        qg_downside_returns = qg_log_returns[qg_log_returns < 0]
        bh_downside_returns = bh_log_returns[bh_log_returns < 0]
        qg_rolling_downside_dev = qg_downside_returns.rolling(window=WINDOW).std()
        bh_rolling_downside_dev = bh_downside_returns.rolling(window=WINDOW).std()
        qg_annualized_downside_dev = qg_rolling_downside_dev * np.sqrt(ANNUAL_TRADING_DAYS)
        bh_annualized_downside_dev = bh_rolling_downside_dev * np.sqrt(ANNUAL_TRADING_DAYS)
        qg_rolling_sortino = qg_rolling_mean_ret / qg_annualized_downside_dev
        bh_rolling_sortino = bh_rolling_mean_ret / bh_annualized_downside_dev

        # Rasio Keuangan
        epsilon = 1000000
        # Menggunakan market_value_safe untuk menghindari pembagian nol yang menyebabkan NaN pada Plot 3 & 4
        market_value_safe = np.where(market_value_array == 0, epsilon, market_value_array)

        qg_equity_to_market_value = quantxi_equity_array / market_value_safe
        qg_leverage = market_value_safe / quantxi_equity_array # Menggunakan safe MV juga di sini

        dates_for_returns = dates[1:]
        dates_for_volatility = dates[WINDOW:]

        # ----------------------------------------------------------------------------------
        # Plot 2: Daily Log Returns (Visualisasi Volatilitas Buy & Hold)
        # ----------------------------------------------------------------------------------
        # ax2.plot(dates_for_returns, bh_log_returns * 100, label='Buy & Hold Daily Log Return', color='red', alpha=0.7)
        # ax2.set_title('Daily Log Returns (Visualisasi Volatilitas Buy & Hold)', fontsize=14)
        # ax2.set_ylabel('Daily Log Return (%)', fontsize=10)
        # ax2.tick_params(axis='x', rotation=45)
        # ax2.axhline(0, color='gray', linestyle='--', linewidth=0.8)
        # ax2.legend(loc='upper right')
        # ax2.grid(True, linestyle='--', alpha=0.6)
        # ax2.text(0.0, -0.2,
        #         'Catatan: Visualisasi volatilitas harian Buy & Hold. Semakin besar ayunan, semakin tinggi risiko.\nVisualisasi volatilitas harian Buy & Hold. Semakin besar ayunan, semakin tinggi risiko.\nVisualisasi volatilitas harian Buy & Hold. Semakin besar ayunan, semakin tinggi risiko.',
        #         transform=ax2.transAxes, fontsize=9, color='black', ha='left')

        # ----------------------------------------------------------------------------------
        # Plot 3: Equity to Market Value Ratio
        # ----------------------------------------------------------------------------------
        ax3.plot(dates, qg_equity_to_market_value * 100, label='QG Equity/MV (%)', color='magenta', linewidth=2)

        ax3.axhline(0, color='black', linestyle='-', linewidth=1.5, label='0% (Bangkrut)')
        ax3.axhline(25, color='orange', linestyle='--', linewidth=1.0, label='25% (Margin Minimum)')
        ax3.axhline(100, color='green', linestyle='-', linewidth=1.5, alpha=0.8, label='100% (Tanpa Utang)')
        ax3.set_title('Equity to Market Value Ratio (Modal vs Total Aset)', fontsize=14)
        ax3.set_ylabel('Rasio E/MV (%)', fontsize=10)
        ax3.tick_params(axis='y', labelcolor='black')
        ax3.grid(True, linestyle='--', alpha=0.6)
        ax3.legend(loc='best')
        ax3.text(0.0, -0.2,
                'Catatan: Rasio modal (Equity) terhadap total aset (MV). Nilai < 100% menunjukkan penggunaan leverage.\nRasio modal (Equity) terhadap total aset (MV). Nilai < 100% menunjukkan penggunaan leverage.\nRasio modal (Equity) terhadap total aset (MV). Nilai < 100% menunjukkan penggunaan leverage.',
                transform=ax3.transAxes, fontsize=9, color='black', ha='left')


        # ----------------------------------------------------------------------------------
        ## Plot 4: Rasio Leverage Harian (Dipindah dari ax3)
        # ----------------------------------------------------------------------------------
        # ax4.plot(dates, qg_leverage, label='Rasio Leverage Aktual', color='purple', linewidth=2)
        # ax4.axhline(1.0, color='green', linestyle='--', linewidth=1.5, alpha=0.8, label='Leverage 1x (Tanpa Utang)')
        # ax4.axhline(2.0, color='orange', linestyle=':', linewidth=1.5, alpha=0.8, label='Leverage 2x')
        # ax4.set_title('Rasio Leverage Aktual Harian QuantGenius', fontsize=14)
        # ax4.set_ylabel('Rasio Leverage (x)', fontsize=10)
        # ax4.tick_params(axis='y', labelcolor='black')
        # ax4.grid(True, linestyle='--', alpha=0.6)
        # ax4.legend(loc='upper right')
        # ax4.text(0.0, -0.2,
        #         'Catatan: Menunjukkan tingkat leverage (MV/E) yang digunakan QG. Nilai > 1.0 berarti menggunakan utang.\nMenunjukkan tingkat leverage (MV/E) yang digunakan QG. Nilai > 1.0 berarti menggunakan utang.\nMenunjukkan tingkat leverage (MV/E) yang digunakan QG. Nilai > 1.0 berarti menggunakan utang.',
        #         transform=ax4.transAxes, fontsize=9, color='black', ha='left')

        # ----------------------------------------------------------------------------------
        # Plot 5: Rolling Sharpe Ratio (Dipindah dari ax4)
        # ----------------------------------------------------------------------------------
        # ax5.plot(dates_for_volatility,
        #         qg_rolling_sharpe.iloc[WINDOW-1:],
        #         label=f'QuantGenius (Rolling {WINDOW}d)',
        #         color='blue')

        # ax5.plot(dates_for_volatility,
        #         bh_rolling_sharpe.iloc[WINDOW-1:],
        #         label=f'Buy & Hold (Rolling {WINDOW}d)',
        #         color='red',
        #         linestyle='--')

        # ax5.axhline(0.0, color='gray', linestyle='--', linewidth=0.8)
        # ax5.set_title(f'Rolling Sharpe Ratio Tahunan ({WINDOW} Hari)', fontsize=14)
        # ax5.set_ylabel('Sharpe Ratio', fontsize=10)
        # ax5.tick_params(axis='x', rotation=45)
        # ax5.legend()
        # ax5.grid(True, linestyle='--', alpha=0.6)
        # ax5.text(0.0, -0.2,
        #         'Catatan: Mengukur return berbanding risiko (volatilitas) yang disetahunkan. Nilai > 1.0 dianggap kinerja baik.\nMengukur return berbanding risiko (volatilitas) yang disetahunkan. Nilai > 1.0 dianggap kinerja baik.\nMengukur return berbanding risiko (volatilitas) yang disetahunkan. Nilai > 1.0 dianggap kinerja baik.',
        #         transform=ax5.transAxes, fontsize=9, color='black', ha='left')

        # ----------------------------------------------------------------------------------
        # Plot 6: Rolling Sortino Ratio (Dipindah dari ax5)
        # ----------------------------------------------------------------------------------
        # ax6.plot(dates_for_volatility,
        #         qg_rolling_sortino.iloc[WINDOW-1:],
        #         label=f'QuantGenius (Rolling {WINDOW}d)',
        #         color='green')

        # ax6.plot(dates_for_volatility,
        #         bh_rolling_sortino.iloc[WINDOW-1:],
        #         label=f'Buy & Hold (Rolling {WINDOW}d)',
        #         color='red',
        #         linestyle='--')

        # ax6.axhline(0.0, color='gray', linestyle='--', linewidth=0.8)
        # ax6.set_title(f'Rolling Sortino Ratio Tahunan ({WINDOW} Hari)', fontsize=14)
        # ax6.set_ylabel('Sortino Ratio', fontsize=10)
        # ax6.tick_params(axis='x', rotation=45)
        # ax6.legend()
        # ax6.grid(True, linestyle='--', alpha=0.6)
        # ax6.text(0.0, -0.2,
        #         'Catatan: Mengukur return berbanding risiko downside (kerugian) yang disetahunkan. Fokus pada risiko kerugian.\nMengukur return berbanding risiko downside (kerugian) yang disetahunkan. Fokus pada risiko kerugian.\nMengukur return berbanding risiko downside (kerugian) yang disetahunkan. Fokus pada risiko kerugian.',
        #         transform=ax6.transAxes, fontsize=9, color='black', ha='left')

        # ----------------------------------------------------------------------------------
        # Plot 7: Volatilitas Tahunan Bergulir (Dipindah dari ax6)
        # ----------------------------------------------------------------------------------
        # ax7.plot(dates_for_volatility,
        #         qg_rolling_volatility.iloc[WINDOW-1:] * 100,
        #         label=f'QuantGenius (Rolling {WINDOW}d)',
        #         color='blue')

        # ax7.plot(dates_for_volatility,
        #         bh_rolling_volatility.iloc[WINDOW-1:] * 100,
        #         label=f'Buy & Hold (Rolling {WINDOW}d)',
        #         color='red',
        #         linestyle='--')

        # ax7.set_title(f'Volatilitas Tahunan Bergulir ({WINDOW} Hari)', fontsize=14)
        # ax7.set_ylabel('Volatilitas Tahunan (%)', fontsize=10)
        # ax7.tick_params(axis='x', rotation=45)
        # ax7.legend()
        # ax7.grid(True, linestyle='--', alpha=0.6)
        # ax7.text(0.0, -0.2,
        #         'Catatan: Mengukur risiko (standar deviasi) pengembalian yang disetahunkan. Kinerja yang lebih halus adalah target.\nMengukur risiko (standar deviasi) pengembalian yang disetahunkan. Kinerja yang lebih halus adalah target.\nMengukur risiko (standar deviasi) pengembalian yang disetahunkan. Kinerja yang lebih halus adalah target.',
        #         transform=ax7.transAxes, fontsize=9, color='black', ha='left')

        # ----------------------------------------------------------------------------------
        ## Plot 8: Drawdown Kumulatif (Dipindah dari ax7)
        # ----------------------------------------------------------------------------------
        qg_cumulative_return = np.insert(quantxi_equity_array / initial_eq, 0, 1.0)
        qg_peak = np.maximum.accumulate(qg_cumulative_return)
        qg_drawdown_plot = (qg_peak - qg_cumulative_return) / qg_peak

        bh_cumulative_return = np.insert(buyhold_equity_array / initial_eq, 0, 1.0)
        bh_peak = np.maximum.accumulate(bh_cumulative_return)
        bh_drawdown_plot = (bh_peak - bh_cumulative_return) / bh_peak

        plot_dates = np.insert(dates.values, 0, dates[0] - pd.Timedelta(days=1))

        ax8.plot(plot_dates, qg_drawdown_plot * 100, label='QG Drawdown', color='blue')
        ax8.plot(plot_dates, bh_drawdown_plot * 100, label='B&H Drawdown', color='red')
        ax8.fill_between(plot_dates, qg_drawdown_plot * 100, color='blue', alpha=0.1)
        ax8.fill_between(plot_dates, bh_drawdown_plot * 100, color='red', alpha=0.1)
        ax8.set_title('Drawdown Kumulatif', fontsize=14)
        ax8.set_xlabel('Tanggal', fontsize=10)
        ax8.set_ylabel('Drawdown (%)', fontsize=10)
        ax8.legend()
        ax8.grid(True, linestyle='--', alpha=0.6)
        ax8.text(0.0, -0.45,
                'Catatan: Kerugian dari puncak tertinggi (peak) yang dialami strategi. Semakin kecil dan pendek durasi, semakin baik.\nKerugian dari puncak tertinggi (peak) yang dialami strategi. Semakin kecil dan pendek durasi, semakin baik.\nKerugian dari puncak tertinggi (peak) yang dialami strategi. Semakin kecil dan pendek durasi, semakin baik.',
                transform=ax8.transAxes, fontsize=9, color='black', ha='left')

        # Wrapper untuk menggabungkan dalam satu baris
        wrapper_html = f"""
        <div style="display: flex; gap: 20px; align-items: flex-start; margin: 20px 0;">
            <div style="flex: 1;">
                {styled_df.to_html()}
            </div>
            <div style="flex: 1;">
                {html_code}
            </div>
        </div>
        """

        # Setelah loop selesai, hapus progress bar dan tampilkan hasil
        clear_output(wait=True)
        display(centered_reset_button)
        display(HTML(wrapper_html))
        # print("\n" + "="*50)
        plt.tight_layout()
        plt.show()

        # Tampilkan Tabel Metrik
        # metrics_df = pd.DataFrame({
        #     "Metrik": ["Total Return", "Max Drawdown", "Sharpe Ratio", "Sortino Ratio"],
        #     "QuantGenius": [
        #         f"{metrics_qg['Total Return']:.2%}",
        #         f"{metrics_qg['Max Drawdown']:.2%}",
        #         f"{metrics_qg['Sharpe Ratio']:.2f}",
        #         f"{metrics_qg['Sortino Ratio']:.2f}"
        #     ],
        #     "Buy & Hold": [
        #         f"{metrics_bh['Total Return']:.2%}",
        #         f"{metrics_bh['Max Drawdown']:.2%}",
        #         f"{metrics_bh['Sharpe Ratio']:.2f}",
        #         f"{metrics_bh['Sortino Ratio']:.2f}"
        #     ]
        # })

        # print("\n" + "="*50)
        # print("📊 HASIL PERBANDINGAN PERFORMA")
        # print("="*50 + "\n")
        # display(metrics_df)

        # # Tampilkan ringkasan numerik
        # print("\n" + "="*50)
        # final_equity_qg = quantxi_equity_array[-1] if len(quantxi_equity_array) > 0 else initial_eq
        # final_equity_bh = buyhold_equity_array[-1] if len(buyhold_equity_array) > 0 else initial_eq

        # print(f"Rasio Ekuitas QG/BH (Akhir): {final_equity_qg / final_equity_bh:.4f}")
        # print(f"Ekuitas QG Akhir: {final_equity_qg:,.2f}")
        # print(f"Ekuitas Buy & Hold Akhir: {final_equity_bh:,.2f}")
        # print(f"Likuiditas Berlebih Minimum: {min_excessLiquidity:,.2f}")
        # print(f"Total Iterasi: {dataIdx}")
        # print("\nSimulasi selesai! Klik 'Reset' untuk kembali ke tampilan awal.")

# 3. Logika reset
def reset_output(b):
    with simulation_output:
        clear_output()
        start_button.disabled = False

def on_start_click(b):
    """Fungsi yang dieksekusi saat tombol diklik."""
    with simulation_output:
      try:
          # print("Mencoba menjalankan simulasi...")
          # Panggil fungsi utama simulasi Anda
          start_simulation(test_data, dataHarga, tanggal)
          # print("Simulasi selesai dan plot berhasil dibuat.")
      except ValueError as e:
          # Menangkap error (misalnya: Data kurang dari 1000)
          print(f"\n[❌ SIMULASI GAGAL] {e}")
          print("Tindakan: Silakan perbaiki data input dan klik tombol START lagi.")

# Hubungkan tombol dengan fungsi
start_button.on_click(on_start_click)
reset2_button.on_click(reset_output)

footer_text1 = widgets.HTML(
    value="<p style='text-align: center;line-height: 1.35;margin-top: 2px; font-size: 11.5px'>I am commitment for realistic trading simulation also open source and transparant testing to prove that QuantGenius actually works. Feel free to modify the code to suit your specific testing methodology. You also can test using jupyter notebook or google colab and kaggle or you can create your own testing system by using our api docs. I am commitment for realistic trading simulation also open source and transparant testing to prove that QuantGenius actually works. Feel free to modify the code to suit your specific testing methodology. You also can test using jupyter notebook or google colab and kaggle or you can create your own testing system by using our api docs. Check details in our Github.</p>"
)

centered_footer = widgets.HBox([footer_text1],layout=widgets.Layout(width='95%'))

# 4. Tampilkan semua widget dalam satu tata letak
main_box = widgets.VBox([ai_logo, title_text, main_accordion,start_button, simulation_output, centered_footer, centered_badges], layout=widgets.Layout(align_items='center', border='2px solid #667eea', padding='20px', margin='auto', width='90%'))

display(main_box)

